In [1]:
import pandas as pd
from keras import Input
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle

from tensorflow.python.layers.core import dense

In [2]:
df = pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Drop irrelevant features

In [3]:
df=df.drop(['RowNumber','CustomerId','Surname'], axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Apply encoding to categorical variable 'gender,geo'

In [4]:
label_encoder_gender=LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [5]:
one_hot_encoder_geo = OneHotEncoder()
geo_encoded= one_hot_encoder_geo.fit_transform(df[['Geography']])
geo_encoded

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [6]:
one_hot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


Merge encoded (OHE) with orignal

In [8]:
df=pd.concat([df.drop('Geography', axis=1), geo_encoded_df], axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


save encoders and scalers

In [9]:
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('one_hot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder_geo, file)

Divide data into independent and dependent

In [10]:
X = df.drop('Exited', axis=1)
y= df['Exited']

Split data in train test

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

ANN implementation

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import datetime

Build ANN model

In [14]:
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),   #HL1 connected to input layer
    Dense(32, activation='relu'),   # HL2
    Dense(1, activation='sigmoid')    #Output layer
])

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

Compile Model

In [16]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [17]:
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

Setup Tensorboard

In [18]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callbacks = TensorBoard(log_dir, histogram_freq=1)

Early Stooping

In [19]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

Train Model

In [20]:
history = model.fit(
    X_train, y_train, validation_data = [X_test, y_test], epochs = 100,
    callbacks= [tensorflow_callbacks, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 887us/step - accuracy: 0.8369 - loss: 0.3980 - val_accuracy: 0.8440 - val_loss: 0.3717
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 507us/step - accuracy: 0.8526 - loss: 0.3591 - val_accuracy: 0.8550 - val_loss: 0.3462
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 562us/step - accuracy: 0.8585 - loss: 0.3488 - val_accuracy: 0.8595 - val_loss: 0.3471
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step - accuracy: 0.8600 - loss: 0.3450 - val_accuracy: 0.8565 - val_loss: 0.3414
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 486us/step - accuracy: 0.8586 - loss: 0.3426 - val_accuracy: 0.8555 - val_loss: 0.3548
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 480us/step - accuracy: 0.8606 - loss: 0.3401 - val_accuracy: 0.8595 - val_loss: 0.3434
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step - accuracy: 0.8668 - loss: 0.3335 - val_accuracy: 0.8630 - val_loss: 0.3534
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 481us/step - accuracy: 0.8637 - loss: 0

Save model

In [21]:
model.save('model.keras')

Load tensorboard extension

In [22]:
%load_ext tensorboard

In [26]:

%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 12357), started 0:13:08 ago. (Use '!kill 12357' to kill it.)